In [0]:
import json, math, time, re
from pyspark.sql.functions import col, lit, coalesce, substring_index, when

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS GOLD")
spark.sql("CREATE SCHEMA IF NOT EXISTS GOLD.SPOTIFY")
spark.sql("CREATE TABLE IF NOT EXISTS GOLD.SPOTIFY.spotify_analise")

df_silver = spark.table("silver.spotify.spotify_analise_silver")

df_quase_gold = df_silver.withColumn(
    "spotify_track_uri",
    substring_index("spotify_track_uri", ":", -1)
)

df_gold = df_silver.select(
    "spotify_track_uri",
    "ts",
    "ms_played",
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "reason_start",
    "skipped",
    "year",
    "month",
    "hour",
    "minutes_played",
)

In [0]:
df_estilos = df_gold.select(
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name"
)

df_estilos = df_estilos.withColumn("style", lit("unknown"))

In [0]:
keys = [
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name"
]


df_estilos = (
    df_estilos
    .select(*keys, col("style").alias("style"))
    .dropDuplicates(keys)
)


(
    df_estilos_com_style
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("gold.spotify.spotify_estilos")
)

(
    df_gold_com_style
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("gold.spotify.spotify_analise")
)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

df = spark.table(
    "gold.spotify.spotify_estilos"
)

df_with_id = df.withColumn(
    "id_musica",
    monotonically_increasing_id()
)

display(
    df_with_id
)